In [5]:
!pip install selenium

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd

In [7]:
#실험
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time
import random as r

URL = 'https://www.uniqlo.com/kr/ko/products/E465185-000/00/reviews'
OFFSET_PX    = 500

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(), options=options)
wait = WebDriverWait(driver, 10)
driver.get(URL)

prev_height = driver.execute_script(
    "return document.documentElement.scrollHeight")

while True:
    driver.execute_script("""
        const offset = arguments[0];
        const doc    = document.documentElement;
        window.scrollTo(0, doc.scrollHeight - window.innerHeight - offset);
    """, OFFSET_PX)

    try:
        new_height = wait.until(
        lambda d, h=prev_height:
            (nh := d.execute_script(
                "return document.documentElement.scrollHeight")) > h and nh)
        prev_height = new_height 

    except TimeoutException:
        print("데이터 로딩 완료")
        break

html = driver.page_source

driver.quit()

KeyboardInterrupt: 

In [ ]:
soup = BeautifulSoup(html, "html.parser")

container = soup.select_one(
    "#page-content > div:nth-child(2) > div > div:nth-child(3) "
    "> div > div.layout-container"
)

cards = container.select("div > ul > li > div")

result = []

for card in cards:
    title = card.select_one("div.add-word-break").get_text(strip=True)
    date = card.select_one("div.ito-gray400.ito-margin-left-24").get_text(strip=True)
    size = card.select_one("div:nth-child(3) > p:nth-child(1)").get_text(strip=True).split(':')[-1].strip()
    color = card.select_one("div:nth-child(3) > p:nth-child(2)").get_text(strip=True).split(':')[-1].strip()
    fit = card.select_one("div:nth-child(3) > p:nth-child(3)").get_text(strip=True).split(':')[-1].strip() if card.select_one("div:nth-child(3) > p:nth-child(3)") else None
    review = card.select_one("div.add-word-break.ito-margin-bottom-16 > div").get_text(strip=True)
        
    gender = age_group = height = weight = shoe_size = None
    info_div = card.select_one("div.reviewUserInfo")
    if info_div and info_div.get_text(strip=True) != "선택하지 않음":
        for span in info_div.select("span"):
            t = span.get_text(strip=True)

            if t in ("남성", "여성"):
                gender = t

            elif re.match(r"^\d+대(?:이상)?$", t):
                age_group = t

            elif t.startswith("키"):
                parts = t.split(":", 1)
                height = parts[1].strip() if len(parts) == 2 else None

            elif t.startswith("몸무게"):
                parts = t.split(":", 1)
                weight = parts[1].strip() if len(parts) == 2 else None

            elif "신발" in t:
                parts = t.split(":", 1)
                shoe_size = parts[1].strip() if len(parts) == 2 else None
    
    
    result.append({
    'title' : title,
    'date' : date,
    'size' : size,
    'color' : color,
    'fit' : fit,
    'review' : review,
    'gender' : gender,
    'age_group' : age_group,
    'height' : height,
    'weight' : weight,
    'shoe_size' : shoe_size,
    })

with open('./uniqlo.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4) 

NameError: name 'BeautifulSoup' is not defined